In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'books'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 10766 palavras!


In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [3]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

#Converte matrizes esparsas para np arrays, para os cálculos da rede neural
Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
#print(new_vocabulary)

Número de features antes do chi-quadrado: 10766
----------------------------------------
Número de features após chi-quadrado: 722


In [5]:
def sigmoid(z):
    
    z = 1/(1+np.exp(-z))
    
    return z

In [6]:
def sigmoidGradient(z):

    g = np.zeros(z.shape)
    g = sigmoid(z)*(1 - sigmoid(z))    

    return g

In [7]:
import scipy.optimize

def funcaoCusto_backp(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, flagreg = False,lambda_reg = 1):
    
    # Extrai os parametros de nn_params e alimenta as variaveis Theta1 e Theta2.
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )

    # Qtde de amostras
    m = X.shape[0]
         
    #Custo
    J = 0
    
    #Theta das redes neurais
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    eps = 1e-15

    
    rotulos = np.zeros((len(y), num_labels), dtype=int) 
    for i in range(len(rotulos)):
        rotulos[i][y[i]] = 1

    a1 = np.insert(X, 0, np.ones(m, dtype=int), axis=1)
    z2 = np.dot(a1, Theta1.T)
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, np.ones(a2.shape[0], dtype=int), axis=1)
    z3 = np.dot(a2, Theta2.T)
    a3 = sigmoid(z3)
    
    
    delta3 = a3 - rotulos
    delta2 = np.dot(delta3, Theta2[:, 1:]) * sigmoidGradient(z2)

    if flagreg == True :
        reg = (lambda_reg/(2*m))*(np.sum(Theta1[:, 1:] ** 2) + np.sum(Theta2[:, 1:] ** 2)) 

        J += reg
    
        delta3 = a3 - rotulos
        delta2 = np.dot(delta3, Theta2[:, 1:]) * sigmoidGradient(z2)

        Theta1_grad[:, 0] = (np.dot(delta2.T, a1)[:, 0]/m) 
        Theta1_grad[:, 1:] = (np.dot(delta2.T, a1)[:, 1:]/m) + (lambda_reg/m)*Theta1[:, 1:]

        Theta2_grad[:, 0] = (np.dot(delta3.T, a2)[:, 0]/m) 
        Theta2_grad[:, 1:] = (np.dot(delta3.T, a2)[:, 1:]/m) + (lambda_reg/m)*Theta2[:, 1:]
    
    else:    
        Theta1_grad = np.dot(delta2.T, a1)/m
        Theta2_grad = np.dot(delta3.T, a2)/m
     
    J = np.sum(-rotulos*np.log(a3+eps) - (1 - rotulos)*np.log(1 - a3+eps))/m

    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)])

    return J, grad

In [8]:
def predicao(Theta1, Theta2, Xval, Yval):

    m = Xval.shape[0] # número de amostras
    num_labels = Theta2.shape[0]
    
    p = np.zeros(m)

    a1 = np.hstack( [np.ones([m,1]), Xval] )
    h1 = sigmoid( np.dot(a1, Theta1.T) )

    a2 = np.hstack( [np.ones([m,1]), h1] ) 
    h2 = sigmoid( np.dot(a2,Theta2.T) )
    
    Ypred = np.argmax(h2,axis=1)
    
    acuracia = np.sum(Ypred==Yval)/len(Yval)   
        
    return Ypred, acuracia

In [9]:
def gridSearch(X, Y ,Xval, Yval):
    
    lambda_rnn = [0.01, 0.1, 10, 100]
    
    bestLambda = lambda_rnn[0]
    bestAcc = 0
    
    for l in lambda_rnn:
        Theta1, Theta2 = redes_neurais(Xtrain, Ytrain, l)
        Ypred, acuracia = predicao(Theta1, Theta2, Xval, Yval)
        if bestAcc < acuracia :
            bestLambda = l
            bestAcc = acuracia
        
    return bestLambda

In [10]:
def redes_neurais (Xtrain, Ytrain, lambda_rnn):

    input_layer_size  = Xtrain.shape[1]  # 20x20 dimensao das imagens de entrada
    hidden_layer_size = int((input_layer_size + 2)*2/3)   
    num_labels = 2          # 10 rotulos, de 1 a 10  
                         #  (observe que a classe "0" recebe o rotulo 10)
    epsilon_init = 0.12
    
    # carregando os pesos da camada 1
    Theta1 =  np.random.RandomState(10).rand(hidden_layer_size, 1 + input_layer_size) * 2 * epsilon_init - epsilon_init
    
    # carregando os pesos da camada 2
    Theta2 = np.random.RandomState(10).rand(num_labels, 1 + hidden_layer_size) * 2 * epsilon_init - epsilon_init
    
    # concatena os pesos em um único vetor
    initial_rna_params = np.concatenate([np.ravel(Theta1), np.ravel(Theta2)])
    
    MaxIter = 5000
    
    #lamba_reg = 1
    
    # Minimiza a funcao de custo
    result = scipy.optimize.minimize(fun=funcaoCusto_backp, x0=initial_rna_params, args=(input_layer_size, hidden_layer_size, num_labels, Xtrain, Ytrain,True,lambda_rnn),  
                    method='TNC', jac=True, options={'maxiter': MaxIter})
    
    # Coleta os pesos retornados pela função de minimização
    nn_params = result.x
    
    # Obtem Theta1 e Theta2 back a partir de rna_params
    Theta1 = np.reshape( nn_params[0:hidden_layer_size*(input_layer_size + 1)], (hidden_layer_size, input_layer_size+1) )
    Theta2 = np.reshape( nn_params[ hidden_layer_size*(input_layer_size + 1):], (num_labels, hidden_layer_size+1) )
    
    return Theta1, Theta2

In [11]:
lambda_rnn = gridSearch(Xtrain, Ytrain, Xval, Yval)
Theta1, Theta2 = redes_neurais(Xtrain, Ytrain, lambda_rnn)
Ypred, acuracia = predicao(Theta1, Theta2, Xval, Yval)
print("Acurácia é "+ str(acuracia*100))

Acurácia é 91.75


In [12]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)


	Revocacao   Precisao   F-medida   Classe
	0.946       0.898      0.922      0
	0.887       0.940      0.913      1
	------------------------------------------------
	0.917       0.919      0.918      Média macro
	0.917       0.917      0.917      Média micro

	Acuracia: 0.917


In [13]:
def curva_aprendizado(X, Y, Xval, Yval, lambda_rnn):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        
        Theta1, Theta2 = redes_neurais(X[:i], Y[:i], lambda_rnn)

        Ypred, acuracia = predicao(Theta1, Theta2, X[:i], Y[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(Theta1, Theta2, Xval, Yval)
        perf_val.append(acuracia)


    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()

In [14]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [15]:
import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold

for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    Xtrain, Xval = X3[train_index, :], X3[test_index, :];
    Ytrain, Yval= Y3[train_index], Y3[test_index];

    Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    Xval = Xval[:, index]
    
    #Converte matrizes esparsas para np arrays, para os cálculos da rede neural
    Xtrain = Xtrain.toarray()
    Xval = Xval.toarray()
    
    lambda_rnn = gridSearch(Xtrain, Ytrain, Xval, Yval)
    Theta1, Theta2 = redes_neurais(Xtrain, Ytrain, lambda_rnn)
    Ypred, acuracia = predicao(Theta1, Theta2, Xval, Yval)
    
    print("Melhor lambda")
    print(lambda_rnn)
    
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults )
    
    k = k + 1
    
kf.mediaFolds( resultados, classes )


-----------
1-fold: 
-----------

Melhor lambda
10

	Revocacao   Precisao   F-medida   Classe
	0.910       0.839      0.873      0
	0.825       0.902      0.862      1
	------------------------------------------------
	0.867       0.870      0.869      Média macro
	0.868       0.868      0.868      Média micro

	Acuracia: 0.868

-----------
2-fold: 
-----------

Melhor lambda
10

	Revocacao   Precisao   F-medida   Classe
	0.880       0.846      0.863      0
	0.840       0.875      0.857      1
	------------------------------------------------
	0.860       0.861      0.860      Média macro
	0.860       0.860      0.860      Média micro

	Acuracia: 0.860

-----------
3-fold: 
-----------

Melhor lambda
10

	Revocacao   Precisao   F-medida   Classe
	0.935       0.940      0.937      0
	0.940       0.935      0.938      1
	------------------------------------------------
	0.938       0.938      0.938      Média macro
	0.938       0.938      0.938      Média micro

	Acuracia: 0.938

------